In [ ]:
import cv2
import numpy as np
import tensorflow as tf
import os


MODEL_PATH = "digit.h5"
BOX_SIZE = 250  

# Load Model
if not os.path.exists(MODEL_PATH):
    print(f" Error: '{MODEL_PATH}' not found.")
    exit()

try:
    model = tf.keras.models.load_model(MODEL_PATH)
    print(f" Model Loaded")
except:
    print(" Failed ")
    exit()

# Initialize Webcam
cap = cv2.VideoCapture(0)
cap.set(3, 1280)
cap.set(4, 720)



while True:
    ret, frame = cap.read()
    if not ret: break

    frame = cv2.flip(frame, 1) # Mirror view

   
    h, w, _ = frame.shape
    x1, y1 = int(w/2 - BOX_SIZE/2), int(h/2 - BOX_SIZE/2)
    x2, y2 = x1 + BOX_SIZE, y1 + BOX_SIZE

    
    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
    
   
    roi = frame[y1:y2, x1:x2]
    
    if roi.size > 0:
      
        
        # Grayscale
        gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        
       
        blurred = cv2.GaussianBlur(gray, (5, 5), 0)
        
       
       
        thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                                      cv2.THRESH_BINARY_INV, 11, 2)
        
      
        kernel = np.ones((3, 3), np.uint8)
       
        thresh = cv2.dilate(thresh, kernel, iterations=1) 
        
        
        small_roi = cv2.resize(thresh, (20, 20), interpolation=cv2.INTER_AREA)
        final_img = cv2.copyMakeBorder(small_roi, 4, 4, 4, 4, cv2.BORDER_CONSTANT, value=0)
        
       
        input_data = final_img.reshape(1, 28, 28, 1).astype('float32') / 255.0

        # 3. Predict
        prediction = model.predict(input_data, verbose=0)
        digit = np.argmax(prediction)
        confidence = np.max(prediction)

        # 4. Display Logic
        if confidence > 0.5:
            text = f"{digit} ({int(confidence*100)}%)"
            color = (0, 255, 0)
        else:
            text = "?"
            color = (0, 0, 255)

        # Draw text
        cv2.putText(frame, text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.2, color, 3)

      
        debug_view = cv2.resize(final_img, (200, 200), interpolation=cv2.INTER_NEAREST)
        cv2.imshow("Model Input (Must be White on Black)", debug_view)

    cv2.imshow("Digit Recognizer", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

c:\Users\ASUS\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


✅ Model Loaded. Waiting for camera...

--- INSTRUCTIONS ---
1. Use a THICK BLACK MARKER on white paper.
2. Ensure good lighting (no shadows inside the green box).
3. Look at the 'Model Input' window - it must show a CLEAR WHITE digit.
--------------------
